This notebook aims to test different registration methods using the Ground Truth points that were manually detected on EM and LM images.

#### Loading the Ground truth files in XML

In [35]:
import os
import numpy as np
import bigfish.stack as stack
import bigfish.plot as plot
from utils import xml_to_dataframe


Load the locations for fiducial particles in EM and LM. The locations were manually detected using ICY software and were saved in XML format. These point locations are considered to be the ground truth. For further processing we need to load the XML format and convert it into Pandas dataframe.

In [ ]:
target_xml = 'Ground_truth_fiducials_EM.xml'
target_df = xml_to_dataframe(target_xml)
print(target_df)

source_xml = 'Ground_truth_fiducials_LM.xml'
source_df_small = xml_to_dataframe(source_xml)
print(source_df_small)

EM and LM images have most probably different sizes. LM image is usually much smaller. The points that were detected in LM image coordinate system needs to be rescaled to EM image coordinate system, otherwise it might be very hard with the registration.

In [26]:
def find_the_scale(EM_shape, LM_shape):
    scale_x = EM_shape[0]/LM_shape[0]
    scale_y = EM_shape[1]/LM_shape[1]
    return scale_x, scale_y

In [ ]:
# load the EM and LM images

input_path = "E:/DATA/AI4Life_Pr26/20240805_Trial_data_fiducial_particles/240723_JB294_CLEM-AI4life_sample1/pos1"

EM_image_path = os.path.join(input_path, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_bin4_EM.tif")
LM_image_path  = os.path.join(input_path, "240726_JB295_HEK293_CLEM_LAMP1-488_Particles-555_grid4_pos1_LM.tif")

EMimage = stack.read_image(EM_image_path)
LMimage_small = stack.read_image(LM_image_path)


# Find what is the scaling rate between the 2 images
scale_x, scale_y = find_the_scale(EMimage.shape, LMimage_small.shape)

print("Scale x: ",scale_x)
print("Scale y: ",scale_y)


# Resize the LM point positions
source_df = xml_to_dataframe(source_xml)
source_df['pos_x'] = source_df_small['pos_x']*scale_x
source_df['pos_y'] = source_df_small['pos_y']*scale_y


# Resize the LM image to fit the position of the resized points
LMimage = stack.resize_image(LMimage_small, EMimage.shape, method='bilinear')

#scale_x = 24.873456790123456
#scale_y = 22.966165413533833

Converting the Panda dataframe into numpy array (2D points (X,Y) and 3D points (X,Y,Z)). The 3D points are needed if we plan to convert them to point clouds later

In [29]:
# Converting dataframe into numpy array of coordinate pairs (X,Y) 
target = target_df[['pos_x', 'pos_y']].to_numpy()
source = source_df[['pos_x', 'pos_y']].to_numpy()
source_small = source_df_small[['pos_x', 'pos_y']].to_numpy()

# Adding the Z-dimension to the 2D points
target3D = np.hstack((target, np.zeros((len(target), 1))))
source3D = np.hstack((source, np.zeros((len(source), 1))))
source3D_small = np.hstack((source_small, np.zeros((len(source_small), 1))))